# Line data analysis

---

Data is collected from 8 group chat in line
Downloaded into a txt format provided by the application
The overall process of the data is:
- initialize spark
- clean up data
- analyze

In [2]:
#Necessary for pointing to correct python path
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
#Initialize Spark
import pyspark as ps

spark = ps.sql.SparkSession.builder.master('local[4]').appName('Line Analyser').getOrCreate()

sc = spark.sparkContext

In [4]:
import findspark

findspark.init()

# Pre-processing text

---

In [5]:
#Use 2 stopwords and a custom word list

import regex as re
from nltk.corpus import stopwords
import nltk

In [6]:
stopwords_ind = set(nltk.corpus.stopwords.words('indonesian'))
stopwords_eng = set(nltk.corpus.stopwords.words('english'))

In [7]:
words_remove = ['ya', 'gk', 'g', '-', 
                'yg', 's.', 'f', '|',
                '-----', 'ga', 'gue',
                'yaa', 'gw',
              ]

In [8]:
def is_stopwords(text):
    if text not in stopwords_ind and text not in stopwords_eng and text not in words_remove:
        return text

In [9]:
rdd = sc.textFile("foo")\
    .flatMap(lambda x: x.split()).map(lambda x: x.lower()).map(lambda x: is_stopwords(x))

In [10]:
text_list = rdd.collect()
len(text_list)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/C:/Users/ardan/Desktop/Codes_projects/Conda Folder/Line Analysis/foo
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:297)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:239)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:325)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.

In [ ]:
text_list_cleaned = [x for x in text_list if x]

# Analyze

In [ ]:
#Analyze using spark, count occurance of the words
rdd_txt_cl = sc.parallelize(text_list_cleaned).map(lambda word: (word,1))\
.reduceByKey(lambda x, y: x + y)\
.sortBy(lambda amount: amount[1], ascending=False)

In [ ]:
text_freq = rdd_txt_cl.collect()

In [ ]:
text_freq_dict = {
    'text':[],
    'freq':[]
}

In [ ]:
for i in text_freq:
    text_freq_dict['text'].append(i[0])
    text_freq_dict['freq'].append(i[1])

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(text_freq_dict, orient='columns')

In [ ]:
df

In [ ]:
df2 = df.head(15)

In [ ]:
df2.plot.bar(x='text', y='freq', title='Top 15 Text', legend = True, )

As we can observe from initial analysis of the line dataset obtained from 8 different chats.
The total amount of words (post-process) is : 399053
The most recurring word: Dyah